In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd
import numpy as np

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

2194/2194 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
573/573 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step


In [ ]:
train.head()


,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train.describe()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
count,120.000000,120.000000,120.000000,120.000000,120.000000
mean,5.845000,3.065000,3.739167,1.196667,1.000000
std,0.868578,0.427156,1.822100,0.782039,0.840168
min,4.400000,2.000000,1.000000,0.100000,0.000000
25%,5.075000,2.800000,1.500000,0.300000,0.000000
50%,5.800000,3.000000,4.400000,1.300000,1.000000
75%,6.425000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train.shape


(120, 4)

In [ ]:
#Input function
train_y_cat = tf.keras.utils.to_categorical(train_y, num_classes=3)
test_y_cat = tf.keras.utils.to_categorical(test_y, num_classes=3)

def make_input_fn(data_df, labels_df,training=True, batch_size=256): #Это означает, что tf.data.Dataset будет разбит на batch'и
#по 256 строк, которые будут поступать в модель по очереди при обучении (model.fit() или .train()).
 # Convert the inputs to a Dataset.
  ds = tf.data.Dataset.from_tensor_slices((dict(data_df), labels_df))
 # Shuffle and repeat if you are in training mode.
  if training: #training is true
    ds = ds.shuffle(1000).repeat()
  return ds.batch(batch_size)

train_ds = make_input_fn(train, train_y_cat, training=True)
test_ds = make_input_fn(test, test_y_cat, training=False)

In [ ]:
my_feature_columns = []
for key in train.keys():
    normalizer = tf.keras.layers.Normalization()
    normalizer.adapt(np.array(train[key]).reshape(-1, 1))
    my_feature_columns.append((key, normalizer))
print(my_feature_columns)

[('SepalLength', <Normalization name=normalization, built=True>), ('SepalWidth', <Normalization name=normalization_1, built=True>), ('PetalLength', <Normalization name=normalization_2, built=True>), ('PetalWidth', <Normalization name=normalization_3, built=True>)]


In [ ]:
#Input function

inputs = {}
encoded_features = []

for name, feature_layer in my_feature_columns:
    input_ = tf.keras.Input(shape=(1,), name=name)
    encoded = feature_layer(input_)
    inputs[name] = input_
    encoded_features.append(encoded)

concatenated = tf.keras.layers.concatenate(encoded_features)
#все что было сверху я скопипейстила только убрала стринг лоокап потому что у нас все числовые

#DNN model
x = tf.keras.layers.Dense(30, activation='relu')(concatenated)
x = tf.keras.layers.Dense(10, activation='relu')(x)

# 4. Выходной слой — 3 класса, softmax
output = tf.keras.layers.Dense(3, activation='softmax')(x)
#explanation in notebook with drawings

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#это тоже копипейст только здесь поменяла loss


model.fit(
    train_ds,
    steps_per_epoch=5000,  # имитация Estimator
    epochs=1                # всего 1 эпоха, как и Estimator делает
)

# 9. Оценка
result = model.evaluate(test_ds)
print("Accuracy on test set:", result[1])

5000/5000 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9620 - loss: 0.1146
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.9667 - loss: 0.4058
Accuracy on test set: 0.9666666388511658


In [ ]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
        val = input(feature + ": ")
        try:
            float_val = float(val)  # проверка и сразу преобразование
            valid = False           # выходим из цикла
        except ValueError:
            print("Please enter a numeric value.")
  predict[feature] = [float_val]

predict_ds = input_fn(predict, batch_size=1)
predictions = model.predict(predict_ds)
class_id = tf.argmax(predictions[0]).numpy()
probability = predictions[0][class_id]
print('Prediction is "{}" ({:.1f}%)'.format(
    SPECIES[class_id], 100 * probability
))


Please type numeric values as prompted.
SepalLength: 2.4
SepalWidth: 2.2
PetalLength: 1.6
PetalWidth: 1.8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Prediction is "Virginica" (100.0%)
